In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("Data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF_IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# Count Vectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Explore parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate':[0.1]
}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,305.576909,8.706000,0.386029,0.053968,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.979354,0.969452,0.967655,0.969452,0.969822,0.005286,1
5,468.823592,4.492402,0.246553,0.031101,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.977558,0.970350,0.968553,0.964960,0.969283,0.004632,2
3,445.921064,10.492509,0.401432,0.046977,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.968582,0.977558,0.966757,0.967655,0.965858,0.969282,0.004236,3
0,192.954213,2.144599,0.410765,0.058751,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.980251,0.968553,0.964960,0.967655,0.968923,0.005976,4
4,345.956329,17.306651,0.371837,0.034260,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.977558,0.968553,0.965858,0.967655,0.968564,0.004855,5


In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate':[0.1]
}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,460.469023,15.884763,0.455406,0.103845,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.966786,0.978456,0.968553,0.964061,0.971249,0.969821,0.004911,1
1,279.194886,4.330712,0.361125,0.048174,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.979354,0.970350,0.964960,0.967655,0.969282,0.005492,2
5,483.761071,4.636638,0.275880,0.020426,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.976661,0.970350,0.964960,0.970350,0.969103,0.004739,3
0,179.733212,3.691132,0.324248,0.038647,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.976661,0.970350,0.964061,0.968553,0.968744,0.004666,4
4,378.571708,14.019717,0.392566,0.072894,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.962298,0.974865,0.967655,0.964061,0.973046,0.968385,0.004899,5
